In [1]:
import sys
import torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
from _utils import  sample_random_glue_qnli, collect_info_for_metric, save_info, \
                    get_continuation_mapping, get_continuous_attributions, get_continuous_raw_inputs,\
                    attr_normalizing_func
from preload_models import get_qnli_tok_n_model

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
tokenizer, model = get_qnli_tok_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import IntegratedGradients
from captum.attr import visualization 

In [6]:
IG = IntegratedGradients(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution, delta = IG.attribute(input_emb, return_convergence_delta=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = delta
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0
word attr tensor([-0.8266, -0.0755, -0.1022, -0.0013,  0.0597, -0.0912, -0.0623, -0.1611,
         0.0230,  0.0379, -0.0521, -0.0529, -0.0805,  0.2204,  0.2256, -0.0447,
        -0.0401,  0.0015, -0.0046, -0.0786, -0.0065,  0.0114, -0.0688, -0.0568,
        -0.0431,  0.0573,  0.0169,  0.0238,  0.0368,  0.0273,  0.0220, -0.0350,
        -0.0675,  0.0517,  0.0136,  0.0132, -0.0245, -0.0357, -0.0149, -0.0433,
        -0.0807,  0.0142, -0.0010, -0.1188, -0.0040,  0.0311, -0.0034, -0.1154,
        -0.0915, -0.0377,  0.0015, -0.1872, -0.0220, -0.0948, -0.0946, -0.0415,
        -0.0314, -0.0013,  0.0713], dtype=torch.float64,
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.8266, dtype=tor

<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1
word attr tensor([-0.2654,  0.0358, -0.3723, -0.1778, -0.2961, -0.2296, -0.0131,  0.0630,
         0.0578, -0.1615, -0.2656,  0.1674, -0.1523,  0.4414,  0.0076, -0.0258,
        -0.0407,  0.0449,  0.0068, -0.1006, -0.0175, -0.0606, -0.0456, -0.0467,
        -0.0900, -0.0374, -0.0932, -0.0529, -0.1292,  0.0450,  0.0012, -0.1301,
        -0.0266, -0.0347,  0.0375,  0.0476, -0.0633, -0.0810, -0.1126, -0.0080,
        -0.0570, -0.0630, -0.0165, -0.0458, -0.0654,  0.0130,  0.0972, -0.1909,
        -0.0090,  0.1469, -0.075

<IPython.core.display.HTML object>
Raw datum: ['What was the name of the Media Day event for Super Bowl 50?', "The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night."]
GT target: 1
word attr tensor([-1.7699e-01, -2.2420e-02, -1.8818e-01, -4.2588e-02, -2.4182e-01,
        -1.3409e-01,  2.4137e-02,  8.0594e-02,  5.0124e-01,  1.4382e-01,
         1.8378e-02, -6.9783e-02, -5.0109e-02, -1.5125e-01, -1.5599e-01,
         3.6883e-01,  3.4552e-01,  1.4540e-01,  7.7035e-02, -7.1996e-03,
         1.7397e-01, -9.1372e-02,  7.3951e-02, -1.8535e-03, -1.2388e-01,
        -7.7326e-02, -3.7833e-02,  1.5870e-04, -5.7079e-02,  8.9109e-03,
        -1.7387e-02, -1.6622e-01, -2.6392e-03, -6.2756e-02,  4.4716e-02,
         2.7416e-02, -5.3985e-02, -8.1680e-02,  3.8604e-02, -7.5565e-02,
         2.3858e-03, -2.6783e-02, -7.4320e-03, -5.7254e-02,  6.5960e-02,
         1.3022e-01,  8.2074e-02, -1.490

<IPython.core.display.HTML object>
Raw datum: ['How many Doctor Who soundtracks have been released since 2005?', 'The fourth was released on 4 October 2010 as a two disc special edition and contained music from the 2008–2010 specials (The Next Doctor to End of Time Part 2).']
GT target: 0
word attr tensor([-0.7055, -0.0414, -0.2043,  0.0493, -0.0660, -0.0811, -0.0589, -0.0814,
        -0.0744,  0.0491, -0.1898, -0.4041, -0.0880,  0.2511,  0.2619,  0.0677,
         0.0056, -0.1312, -0.0283, -0.0272, -0.0526, -0.0224, -0.0966, -0.0421,
        -0.0183,  0.0544, -0.0275,  0.0042, -0.0392,  0.0472, -0.0365, -0.0314,
        -0.0110, -0.0296,  0.0063, -0.0160, -0.0311, -0.0396,  0.0285,  0.0631,
        -0.0757,  0.0269, -0.0222, -0.0439,  0.0192, -0.0665,  0.0010, -0.0197,
        -0.0310,  0.1311], dtype=torch.float64, grad_fn=<DivBackward0>)
conti attr [tensor(-0.7055, dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0414, dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor

<IPython.core.display.HTML object>
Raw datum: ["What is the name of the country's longest continuously running student film society?", 'Students at the University of Chicago run over 400 clubs and organizations known as Recognized Student Organizations (RSOs).']
GT target: 0
word attr tensor([-0.1743, -0.0030, -0.0907,  0.0188, -0.1505,  0.1185,  0.0018,  0.0244,
        -0.0216, -0.6261, -0.3609,  0.0945, -0.0040,  0.0025, -0.1082, -0.0504,
         0.2193,  0.1935, -0.1806, -0.1005, -0.0896, -0.2258, -0.0172, -0.2341,
         0.0873, -0.0628, -0.1093, -0.0533,  0.0805, -0.1537, -0.0924,  0.0077,
        -0.0363,  0.0329,  0.0763, -0.0607,  0.0780,  0.0115, -0.0042, -0.0321,
         0.2264], dtype=torch.float64, grad_fn=<DivBackward0>)
conti attr [tensor(-0.1743, dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0030, dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0907, dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(0.0188, dtype=torch.float64, grad_fn=

...

<IPython.core.display.HTML object>
Raw datum: ['Who did Britain exploit in India?', 'Although a substantial number of colonies had been designed to provide economic profit and to ship resources to home ports in the seventeenth and eighteenth centuries, Fieldhouse suggests that in the nineteenth and twentieth centuries in places such as Africa and Asia, this idea is not necessarily valid:']
GT target: 0
word attr tensor([-7.5843e-01, -4.9680e-01, -1.0112e-01, -3.1126e-02, -4.0125e-02,
         3.3742e-02, -2.1858e-02, -4.1988e-02,  1.3777e-01,  1.5220e-01,
         2.5061e-02, -8.7414e-03, -1.1773e-01, -1.2556e-01, -1.5077e-02,
         4.6896e-03, -5.2654e-02, -5.0256e-02, -1.9021e-02, -2.7624e-02,
        -1.9759e-02, -1.2736e-02, -8.5371e-03,  1.8331e-02, -1.9761e-02,
         2.2236e-02, -3.7971e-04,  4.2032e-03, -5.1402e-03,  2.1037e-03,
         1.9321e-02, -5.8655e-03, -4.1675e-02, -2.6167e-03,  2.9848e-02,
        -8.1172e-02,  4.5478e-02,  4.0011e-03,  4.3048e-02, -3.0968e-03,


<IPython.core.display.HTML object>
Raw datum: ['What popular environmentalist is also a university alumni member?', 'In science, alumni include astronomers Carl Sagan, a prominent contributor to the scientific research of extraterrestrial life, and Edwin Hubble, known for "Hubble\'s Law", NASA astronaut John M. Grunsfeld, geneticist James Watson, best known as one of the co-discoverers of the structure of DNA, experimental physicist Luis Alvarez, popular environmentalist David Suzuki, balloonist Jeannette Piccard, biologists Ernest Everett']
GT target: 1
word attr tensor([-9.3622e-01,  9.1712e-03,  4.1791e-02,  5.5018e-02,  3.7341e-03,
        -1.6485e-02, -2.3336e-02, -1.8279e-03, -5.9702e-02, -4.6744e-02,
        -6.8895e-02, -2.5662e-02,  1.6898e-01,  1.6113e-01,  1.2030e-03,
         7.5340e-03,  2.2294e-02,  4.2945e-03, -2.5368e-02, -1.5962e-02,
        -1.7005e-03,  5.4042e-03,  2.0370e-02, -1.0646e-03,  1.0915e-02,
        -5.4273e-03, -1.3369e-03, -5.7155e-03,  1.4030e-02,  9.3

<IPython.core.display.HTML object>
Raw datum: ['What publication did Philip Howard work for?', 'Responding to the findings of the survey in The Times newspaper, journalist Philip Howard maintained that, "to compare the violence of Dr Who, sired by a horse-laugh out of a nightmare, with the more realistic violence of other television series, where actors who look like human beings bleed paint that looks like blood, is like comparing Monopoly with the property market in London: both are fantasies, but one is meant to be taken seriously."']
GT target: 1
word attr tensor([-9.7112e-01, -5.5920e-03, -6.6671e-04, -1.9113e-02,  1.1513e-02,
         1.8091e-03, -1.0543e-03,  6.5965e-03,  6.5178e-03,  1.0710e-01,
         1.2890e-01,  1.6087e-02,  2.0903e-03, -1.9408e-02, -2.8768e-02,
        -7.5639e-03,  1.2424e-03, -1.9925e-02,  3.2901e-03,  5.3330e-03,
        -5.5751e-03, -1.2589e-02, -1.5728e-03, -1.3712e-03,  1.7397e-02,
         2.3828e-02, -2.6875e-03, -1.0712e-02, -1.0600e-03,  1.6396e

<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1
word attr tensor([-8.3527e-01, -1.4693e-03, -1.0493e-02,  8.9575e-03, -7.9647e-02,
         1.1714e-02,  2.3719e-02,  2.2750e-01,  8.5093e-02, -6.6104e-02,
         8.4129e-02, -8.7847e-02,  3.8307e-02,  1.3486e-02, -9.0070e-02,
        -6.5520e-02, -8.2740e-02, -2.1587e-02, -2.8930e-02,  3.2655e-02,
         1.4379e-01,  1.4180e-01, -7.0541e-02,  6.4417e-02, -1.0115e-01,
         3.6355e-02, -2.8036e-02,  5.9490e-02, -2.2113e-02, -3.0247e-02,
         1.1983e-02, -6.2856e-02,  2.1694e-02,

<IPython.core.display.HTML object>


In [9]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='integrated_gradients_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho